- Import Packages

In [1]:
import os
import random
import math
import json

- Change csv file of facebook events to jason list

In [2]:
events = []
for line in open('all_from_fb.txt','r').readlines():
    event = {}        
    split_line = line.strip('\n').split(',')
    event['id'] = split_line[0]
    event['predicted_class'] = ""
    event['content_cut'] = split_line[1:]
    events.append(event)
json.dump(events,open('fb_events.json','w'))

- Merge training data to 13 classes

In [3]:
training_data = json.load(open('accupass_events.json','r'))
testing_data = json.load(open('fb_events.json','r'))

class_content_dict = {"arts":[],
                      "business":[],
                      "charity":[],
                      "education":[],
                      "entertainment":[],
                      "fashion":[],
                      "food":[],
                      "health":[],
                      "other":[],
                      "photography":[],
                      "sports":[],
                      "technology":[],
                      "travel":[]}
for data in training_data:
    class_content_dict[data['class']].extend(data['content_cut'])


json.dump(class_content_dict,open('class_content_dict.json','w'))

- Remove numbers and alphabets and calculate TF/IDF

In [4]:
class_content_dict = json.load(open('class_content_dict.json','r'))

class_content_dict_2 = {"arts":[],
                      "business":[],
                      "charity":[],
                      "education":[],
                      "entertainment":[],
                      "fashion":[],
                      "food":[],
                      "health":[],
                      "other":[],
                      "photography":[],
                      "sports":[],
                      "technology":[],
                      "travel":[]}

for class_name, content in class_content_dict.items():
    for keyword in content:
        flag = True
        for word in keyword:
            char = word.encode('unicode_escape').decode('utf8')
            if not (char>='\\u4e00' and char<='\\u9fa5'):
                flag = False
                break
        if flag:
            class_content_dict_2[class_name].append(keyword)


json.dump(class_content_dict_2,open('class_content_dict_2.json','w'))



class_content_dict = json.load(open('class_content_dict_2.json','r'))

keyword_count_dict = {}
keyword_freq_dict = {}
for class_name, keywords in class_content_dict.items():  
    print(class_name)
    keyword_count_dict[class_name] = {}
    for keyword in keywords:
        if keyword in keyword_count_dict[class_name]:
            keyword_count_dict[class_name][keyword] += 1 
        else:
            keyword_count_dict[class_name][keyword] = 1 
            if keyword in keyword_freq_dict:
                keyword_freq_dict[keyword] += 1
            else:
                keyword_freq_dict[keyword] = 1

keyword_tf_dict = {}
keyword_idf_dict = {}
for class_name, keywords in class_content_dict.items():  
    length = len(keywords)
    keyword_tf_dict[class_name] = {}
    for keyword, count in keyword_count_dict[class_name].items():
        keyword_tf_dict[class_name][keyword] = count/length
for keyword,freq in keyword_freq_dict.items():
    keyword_idf_dict[keyword] = math.log10(13/freq)

keyword_tfidf_dict = {}
for class_name, keyword_tf in keyword_tf_dict.items():
    keyword_tfidf_dict[class_name] = {}
    for keyword, tf in keyword_tf.items():
        keyword_tfidf_dict[class_name][keyword] = keyword_tf_dict[class_name][keyword]*keyword_idf_dict[keyword]

json.dump(keyword_tfidf_dict,open('keyword_tfidf_dict.json','w'))

photography
entertainment
other
technology
education
fashion
health
charity
arts
sports
food
business
travel
